In [1]:
import tensorflow as tf
print(f"TensorFlow has access to the following devices:\n{tf.config.list_physical_devices()}")
print(f"TensorFlow version: {tf.__version__}")

TensorFlow has access to the following devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow version: 2.8.0


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import seaborn as sns
import os
import pickle

# Import data

In [2]:
# Tureky soil samples are the soils starting with 'no' 1000 or above
data = pd.read_csv('data.csv') \
            .drop('hhpa', axis=1)
data['VWC'] = data['VWC']/100
print(data.shape) 
data.head()

(20383, 9)


,fold,no,clay,silt,sand,BD,omc,pF,VWC
0,1,55,32.6,43.3,24.1,1.14,4.50,0.841595,0.558
1,1,97,19.8,67.0,13.2,1.14,3.56,1.069393,0.487
2,1,166,27.7,41.6,30.7,1.15,3.00,2.057260,0.394
3,1,1015,45.7,30.4,24.0,0.85,0.49,1.360000,0.620
4,1,1066,13.5,25.5,61.0,1.23,1.27,2.690000,0.250


## EDA

In [ ]:
data.groupby('fold').count()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(18, 10))
sns.boxplot(data=data.iloc[:, 2:])
plt.show()

In [ ]:
data.describe()

In [ ]:
sns.reset_orig()
plt.figure(figsize=(18,12))
corr = data.iloc[:, 2:].corr()
# Getting the Upper Triangle of the co-relation matrix
matrix = np.triu(corr)

sns.heatmap(corr, annot = True, mask=matrix,cmap="YlGnBu")
plt.show()

In [ ]:
# sns.pairplot(data.iloc[:, 2:]);

# Defining functions

In [4]:
from keras_tuner.tuners import RandomSearch 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [10]:
## Make directory for storing saved models
# os.makedirs('model4')

## Define the `bagging` function

In [5]:
def bagging(rawdata):
    """Take bootstrap samples - sampling with replacement"""
    ###generator random sample id with replacement
    idx = pd.DataFrame(rawdata[['no']])
    idx_rmdup = idx.drop_duplicates()
    length = len(idx_rmdup)
    samples = idx_rmdup.sample(length, replace= True)
    # changing the sample id into one dimensional list in order to label sampleid in idx 
    samples_list = samples.transpose().values.tolist()[0] 
    ###going through the sampleid dataframe and marking the sample as training or validation dataset. 
    idx['labels'] = ['training' if x in samples_list else 'validation'  for x in idx['no'] ]
    rawdata_con = pd.concat([rawdata,idx['labels']],axis = 1)
    ###spliting rawdata into training and validation based on label column 
    grouped_rawdata = rawdata_con.groupby('labels')
    training = grouped_rawdata.get_group('training')
    validation = grouped_rawdata.get_group('validation')
    return training,validation

## `build_model` function

In [6]:
def build_model(hp):
    """Build and compile the NN model"""
    model = tf.keras.Sequential([
       tf.keras.layers.Dense(
                        units=hp.Int('units',
                        min_value=2,
                        max_value=14,
                        step=1),
                        activation=hp.Choice("activation", ["relu", "tanh"])
                        ),
        tf.keras.layers.Dense(units=1)
        ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(),
            loss='mse',
            metrics=['mae'])
    return model

## Define `wettodry` and `drytowet` functions

In [7]:
def wettodry(vwc):
    vwc = [0 if i < 0 else i for i in vwc]
    w2d = [vwc[0]]
    for idx in range(1, len(vwc)):
        if vwc[idx] < w2d[idx-1]:
            w2d.append(vwc[idx])
        else:
            w2d.append(w2d[idx-1])
    return w2d

def drytowet(vwc):
    vwc = [0 if i < 0 else i for i in vwc]
    rslt_reverse = vwc[::-1]
    d2w = [rslt_reverse[0]]
    for idx in range(1, len(vwc)):
        if rslt_reverse[idx] > d2w[idx-1]:
            d2w.append(rslt_reverse[idx])
        else:
            d2w.append(d2w[idx-1])
    return d2w[::-1]

## Define `bag_predict`
and return mean and standard deviations

In [8]:
def bag_predict(models, X_test):
    bag_pred = pd.DataFrame()
    for ann in models:
        # print(ann)
        ann_all = tf.keras.models.load_model(ann)
        y_pred = ann_all.predict(X_test)
        w2d = wettodry(y_pred.ravel().tolist())
        d2w = drytowet(y_pred.ravel().tolist())
        # print(w2d==d2w)
        pred_list = [(g + h) / 2 for g, h in zip(w2d, d2w)]
        bag_pred[ann[ann.find('ann'):-3]] = pd.Series(pred_list)

    mean_vwc = bag_pred.mean(axis=1)
    std_vwc = bag_pred.std(axis=1)
    return mean_vwc, std_vwc

## Define `plot_swrc` function

# Bootstrap (finding and fitting best models)

In [11]:
losses = {}

iteration = 100
for bb in range(iteration):
    training, validation = bagging(data)
    ## MODEL 1
    # X_train = training.iloc[:,2:-2]
    # X_valid = validation.iloc[:,2:-2]
    ## MODEL 2
    # X_train = training.loc[:,['clay', 'silt', 'sand','pF']]
    # X_valid = validation.loc[:,['clay', 'silt', 'sand','pF']]
    ## MODEL 3
    # X_train = training.loc[:,['clay', 'silt', 'sand','BD', 'pF']]
    # X_valid = validation.loc[:,['clay', 'silt', 'sand', 'BD', 'pF']]
    ## MODEL 4
    X_train = training.loc[:,['clay', 'silt', 'sand','omc', 'pF']]
    X_valid = validation.loc[:,['clay', 'silt', 'sand', 'omc', 'pF']]

    y_train = training.iloc[:,-2]
    y_valid = validation.iloc[:,-2]

    # scaler = MinMaxScaler() 
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_valid = scaler.transform(X_valid)

    tuner = RandomSearch(
        build_model,    
        objective='val_loss',
        max_trials=5,
        executions_per_trial=1,
        overwrite=True,
        directory='project',
        project_name='HPtuning_'+str(bb))

    early_stop = [tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                    patience=4,
                    restore_best_weights=True)]

    tuner.search(X_train, y_train,
                epochs=25,
                validation_data=(X_valid, y_valid),
                callbacks=early_stop,
                verbose=2)

    print(f"Fitting the best model for iteration {bb+1}......." "\n"
            f"Training Size = {len(training)}, validation Size = {len(validation)}" "\n"
            f"Unique soils in Training = {len(training.no.unique())}" "\n"
            f"Unique soils in Validation = {len(validation.no.unique())}" "\n"
            f"INPUT SHAPE: {X_train.shape[1]}")

    best_hps = tuner.get_best_hyperparameters(2)
    model = build_model(best_hps[0])

    callback = [tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                    patience=15,
                    restore_best_weights=True),
                tf.keras.callbacks.ModelCheckpoint(filepath="model4/ann_"+str(bb)+".h5",
                    monitor="val_loss",
                    save_best_only=True)]
    # use `model.fit()` here to train and save the best model from `tuner.search`
    history = model.fit(X_train, y_train,
                batch_size = 32, epochs = 500,
                callbacks=callback,
                validation_data=(X_valid, y_valid),
                verbose=0)
    losses[bb] = history.history
    print(f"Activation function --- {model.layers[0].activation}""\n"
        f"Number of hidden neurons = {model.layers[0].units}")
    print("Done")

Trial 5 Complete [00h 01m 17s]
val_loss: 0.003532117698341608

Best val_loss So Far: 0.003046780126169324
Total elapsed time: 00h 06m 02s
INFO:tensorflow:Oracle triggered exit
Fitting the best model for iteration 100.......
Training Size = 12859, validation Size = 7524
Unique soils in Training = 151
Unique soils in Validation = 81
INPUT SHAPE: 5
Activation function --- <function tanh at 0x000001D3E7A52E50>
Number of hidden neurons = 12
Done


In [12]:
with open('history_4.pkl', 'wb') as handle:
    pickle.dump(losses, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('best_HPall_b.pkl.pkl', 'rb') as handle:
#     best_parms = pickle.load(handle)

In [13]:
models = ['model4/ann_'+ str(i) + '.h5' for i in range(100)]
hidden_n, activation_fn = [] ,[]
for ann in models:
    model = tf.keras.models.load_model(ann)
    print(model.layers[0].activation, model.layers[0].units)
    # hidden_n.append(model.layers[0].units), activation_fn.append(model.layers[0].activation)

<function tanh at 0x000001D3E7A52E50> 11
<function tanh at 0x000001D3E7A52E50> 13
<function tanh at 0x000001D3E7A52E50> 6
<function relu at 0x000001D3E7A52AF0> 6
<function tanh at 0x000001D3E7A52E50> 12
<function relu at 0x000001D3E7A52AF0> 13
<function relu at 0x000001D3E7A52AF0> 13
<function relu at 0x000001D3E7A52AF0> 12
<function tanh at 0x000001D3E7A52E50> 10
<function relu at 0x000001D3E7A52AF0> 8
<function tanh at 0x000001D3E7A52E50> 13
<function tanh at 0x000001D3E7A52E50> 8
<function relu at 0x000001D3E7A52AF0> 12
<function relu at 0x000001D3E7A52AF0> 13
<function relu at 0x000001D3E7A52AF0> 8
<function tanh at 0x000001D3E7A52E50> 8
<function tanh at 0x000001D3E7A52E50> 3
<function relu at 0x000001D3E7A52AF0> 12
<function relu at 0x000001D3E7A52AF0> 8
<function tanh at 0x000001D3E7A52E50> 14
<function tanh at 0x000001D3E7A52E50> 11
<function tanh at 0x000001D3E7A52E50> 11
<function relu at 0x000001D3E7A52AF0> 12
<function relu at 0x000001D3E7A52AF0> 11
<function relu at 0x0000

# Predict

## Import data and apply Scaler

In [ ]:
file = os.path.abspath('PTF3_data.xlsx')

data2 = pd.read_excel(file,'lab')
data2 = data2.head(1016).iloc[:,:17]
data2.columns, data2.shape

In [ ]:
data2.head()

In [5]:
## Save Scaler
scaler = StandardScaler()
##MODEL 1
# scaler.fit_transform(data.iloc[:,2:-1])
# pickle.dump(scaler, open('ann1_stdscaler.pkl', 'wb'))

##MODEL 2
scaler.fit_transform(data.loc[:,['clay', 'silt', 'sand', 'omc', 'pF']])
pickle.dump(scaler, open('ann4_stdscaler.pkl', 'wb'))

## MODEL 3
# scaler.fit_transform(data.loc[:,['clay', 'silt', 'sand','BD', 'pF']])
# pickle.dump(scaler, open('ann3_stdscaler.pkl', 'wb'))

In [ ]:
scaler = pickle.load(open('ann1_stdscaler.pkl', 'rb'))
# scaler = pickle.load(open('ann3_stdscaler.pkl', 'rb'))

In [ ]:
data.head()

In [ ]:
def create_Xtest(data):
    """enter the dataframe and this function will return the X_test data"""
    pF = pd.Series(np.arange(-1, 6, 0.05), name='pF')
    df = pd.concat([data.drop_duplicates()]*len(pF), ignore_index=True)
    test = pd.concat([df,pF], axis=1)
    return test

In [ ]:
colList = ['no', 'clay', 'silt', 'sand']
df_group = data[colList].groupby(['no'])
test_all = df_group.apply(create_Xtest)

In [ ]:
rand_soil = random.choice(data.no.unique())
print(rand_soil)
soil_test = test_all[test_all['no']==rand_soil]

X_test = scaler.transform(soil_test.iloc[:,1:])

Have to use `bag_predict` and scaler on each soil individually.
Making predicitons on the entire dataset results in wrong results

## Apply `bag_predict`

In [ ]:
models = ['model2/ann_'+ str(i) + '.h5' for i in range(100)]
# mean_vwc, std_vwc = bag_predict(models, X_test)

In [ ]:
result_all = pd.concat([test_all.reset_index(drop=True), pd.Series(mean_vwc, name = 'mean_vwc'),
            pd.Series(std_vwc, name = 'std_vwc')], axis=1)

## Plot

In [ ]:
sns.set(font_scale=1.5, style="ticks")
import random
plt.figure(figsize=(12, 6))
for i in range(2):
    ax = plt.subplot(1, 2, i + 1)
    rand_soil = random.choice(data.no.unique())
    soil_test = test_all[test_all['no']==rand_soil]
    soil_df = data[data['no']==rand_soil]
    X_test = scaler.transform(soil_test.iloc[:,1:])
    mean_vwc, std_vwc = bag_predict(models, X_test)

    ax.plot(soil_df['pF'], soil_df['VWC'], 'o', alpha=0.6, label = 'VWC')
    ax.plot(soil_test['pF'], mean_vwc, '-', label = 'ANN')
    ax.fill_between(soil_test['pF'], mean_vwc+std_vwc, mean_vwc-std_vwc, alpha=0.3)
    ax.set_xlim(0,5)
    ax.legend(prop={'size': 14})
    ax.grid(True,linestyle='--')
    ax.xaxis.set_major_locator(MultipleLocator(1))
    ax.yaxis.set_major_locator(MultipleLocator(.1))
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.set_xlabel('pF',size = 18, weight = 'bold')
    # ax.set_ylabel('VWC'r' [$cm^3 cm^{-3}$]',size = 18, weight = 'bold')
    ax.set_title(rand_soil)

# Miscellaneous

In [ ]:
# import plotly.graph_objects as go
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=soil_df["pF"], y=soil_df["VWC"]/100,
#                         mode='markers',
#                         name='VWC'))
# fig.add_trace(go.Scatter(x=pF, y=mean_vwc,
#                         mode='lines',
#                         name='ANN_all'))
# fig.show()